In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('train.csv',encoding='latin1')

In [3]:
df.head()

,id,loc,v(g),ev(g),iv(g),n,v,l,d,i,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,0,22.0,3.0,1.0,2.0,60.0,278.63,0.06,19.56,14.25,...,17,1,1,0,16.0,9.0,38.0,22.0,5.0,False
1,1,14.0,2.0,1.0,2.0,32.0,151.27,0.14,7.00,21.11,...,11,0,1,0,11.0,11.0,18.0,14.0,3.0,False
2,2,11.0,2.0,1.0,2.0,45.0,197.65,0.11,8.05,22.76,...,8,0,1,0,12.0,11.0,28.0,17.0,3.0,False
3,3,8.0,1.0,1.0,1.0,23.0,94.01,0.19,5.25,17.86,...,4,0,2,0,8.0,6.0,16.0,7.0,1.0,True
4,4,11.0,2.0,1.0,2.0,17.0,60.94,0.18,5.63,12.44,...,7,0,2,0,7.0,6.0,10.0,10.0,3.0,False


In [4]:
df.shape

(101763, 23)

In [5]:
df.isnull().sum()

id                   0
loc                  0
v(g)                 0
ev(g)                0
iv(g)                0
n                    0
v                    0
l                    0
d                    0
i                    0
e                    0
b                    0
t                    0
lOCode               0
lOComment            0
lOBlank              0
locCodeAndComment    0
uniq_Op              0
uniq_Opnd            0
total_Op             0
total_Opnd           0
branchCount          0
defects              0
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101763 entries, 0 to 101762
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 101763 non-null  int64  
 1   loc                101763 non-null  float64
 2   v(g)               101763 non-null  float64
 3   ev(g)              101763 non-null  float64
 4   iv(g)              101763 non-null  float64
 5   n                  101763 non-null  float64
 6   v                  101763 non-null  float64
 7   l                  101763 non-null  float64
 8   d                  101763 non-null  float64
 9   i                  101763 non-null  float64
 10  e                  101763 non-null  float64
 11  b                  101763 non-null  float64
 12  t                  101763 non-null  float64
 13  lOCode             101763 non-null  int64  
 14  lOComment          101763 non-null  int64  
 15  lOBlank            101763 non-null  int64  
 16  lo

In [7]:
unwanted_cols=['id','defects']
cols=[x for x in df.columns if x not in unwanted_cols]

In [9]:
X=df[cols]
Y=df['defects']

In [10]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test=train_test_split(X, Y, test_size=0.3, random_state=42)

In [11]:
df.loc[X_train.index,'Sample']='DEV'
df.loc[X_test.index,'Sample']='VAL'

In [12]:
df['Sample'].value_counts()

DEV    71234
VAL    30529
Name: Sample, dtype: int64

In [224]:
X_train=df[df['Sample']=='DEV']
X_test=df[df['Sample']=='VAL']

In [237]:
X_train_c=X_train[selected_cols]
Y_train=X_train['defects']
X_test_c=X_test[selected_cols]
Y_test=X_test['defects']

In [238]:
import statsmodels.api as sm
logit_model=sm.Logit(Y_train,X_train_c)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.469226
         Iterations 8
                          Results: Logit
Model:               Logit            Pseudo R-squared: 0.124     
Dependent Variable:  defects          AIC:              66879.7270
Date:                2023-10-18 07:14 BIC:              67017.3329
No. Observations:    71234            Log-Likelihood:   -33425.   
Df Model:            14               LL-Null:          -38163.   
Df Residuals:        71219            LLR p-value:      0.0000    
Converged:           1.0000           Scale:            1.0000    
No. Iterations:      8.0000                                       
------------------------------------------------------------------
                   Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
------------------------------------------------------------------
loc                0.0029   0.0004   6.6179 0.0000  0.0020  0.0037
v(g)              -0.0518   0.0088  -5.8580 0.0000 -0

In [233]:
removable_cols=['v','d','b','t','total_Op']
selected_cols=[x for x in cols if x not in removable_cols]

In [236]:
removable_cols=['iv(g)']
selected_cols=[x for x in selected_cols if x not in removable_cols]

In [239]:
X_train_c=X_train[selected_cols]
Y_train=X_train['defects']
X_test_c=X_test[selected_cols]
Y_test=X_test['defects']

In [240]:
def ks(y,pred):
    ksd=pd.DataFrame()
    ksd['target']=list(y) ### list of actual values
    ksd['prob']=list(pred)
    ksd['target0'] = 1 - ksd['target']
    np.random.seed(1)
    ksd['random'] = np.random.random(ksd.shape[0])
    ksd.sort_values(by=['prob','random'],ascending=False,inplace=True)
#print(ksd)
    ksd['bucket'] = pd.qcut(ksd['prob'].rank(method='first'), 10,labels=['1','2','3','4','5','6','7','8','9','10'])
    grouped = ksd.groupby(ksd['bucket'], as_index = False)
    kstable = pd.DataFrame()
    kstable['Total_Count'] = grouped.sum()['target'] + grouped.sum()['target0']
    kstable['min_prob'] = grouped.min()['prob']
    kstable['max_prob'] = grouped.max()['prob']
    kstable['mean_prob'] = grouped.mean()['prob']
    kstable['Target'] = grouped.sum()['target']
    kstable['Target_rate'] = (kstable.Target /( grouped.sum()['target'] + grouped.sum()['target0']) ).apply('{0:.3%}'.format)
    kstable['NonTarget'] = grouped.sum()['target0']
#kstable['nocashout_rate'] = (kstable.nocashout / ksd['target0'].sum()).apply('{0:.3%}'.format)
    kstable = kstable.sort_values(by="min_prob", ascending=False).reset_index(drop = True)
    kstable['cum_Target']=(kstable.Target / ksd['target'].sum()).cumsum()
    kstable['cum_nonTarget']=(kstable.NonTarget / ksd['target0'].sum()).cumsum()
    kstable['cum_pct_Total'] = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
    kstable['KS'] = np.abs(np.round(kstable['cum_Target']-kstable['cum_nonTarget'], 4) * 100)

#Formating
    kstable['cum_Target']= kstable['cum_Target'].apply('{0:.2%}'.format)
    kstable['cum_nonTarget']= kstable['cum_nonTarget'].apply('{0:.2%}'.format)
    kstable['cum_pct_Total']= kstable['cum_pct_Total'].apply('{0:.2%}'.format)
    kstable.index = range(1,11)
    kstable = kstable.astype({ 'Total_Count':int , 'Target' : int , 'NonTarget' : int})
    kstable.index.rename('Bin', inplace=True)
    del kstable['NonTarget']

    print( "KS is " + str(max(kstable['KS'])))
    return(kstable)

In [17]:
import lightgbm as lgbm

In [294]:
clf=lgbm.LGBMClassifier(importance_type='gain',n_estimators=150,max_depth=5,random_state=50,learning_rate=0.05,num_leaves=25)

In [242]:
clf.fit(X_train_c,Y_train)

LGBMClassifier(importance_type='gain', learning_rate=0.05, max_depth=5,
               n_estimators=150, num_leaves=25, random_state=50)

In [243]:
y_pred=clf.predict_proba(X_train_c)[:,1]

In [244]:
ks(Y_train,y_pred)

KS is 46.7


,Total_Count,min_prob,max_prob,mean_prob,Target,Target_rate,cum_Target,cum_nonTarget,cum_pct_Total,KS
Bin,,,,,,,,,,
1,7124,0.579293,0.901494,0.692540,5117,71.828%,31.63%,3.65%,10.00%,27.99
2,7123,0.375915,0.579287,0.473818,3384,47.508%,52.55%,10.44%,20.00%,42.11
3,7123,0.251452,0.375910,0.306964,2191,30.760%,66.09%,19.39%,30.00%,46.70
4,7124,0.180186,0.251433,0.213138,1531,21.491%,75.56%,29.55%,40.00%,46.00
5,7123,0.139509,0.180181,0.158358,1108,15.555%,82.41%,40.48%,50.00%,41.93
6,7123,0.111515,0.139508,0.124857,898,12.607%,87.96%,51.78%,60.00%,36.17
7,7124,0.095005,0.111494,0.103180,737,10.345%,92.51%,63.39%,70.00%,29.13
8,7123,0.071271,0.095005,0.083883,523,7.342%,95.75%,75.37%,80.00%,20.37
9,7123,0.056665,0.071266,0.063391,427,5.995%,98.39%,87.53%,90.00%,10.85


In [245]:
y_pred_test=clf.predict_proba(X_test_c)[:,1]

In [246]:
ks(Y_test,y_pred_test)

KS is 44.32


,Total_Count,min_prob,max_prob,mean_prob,Target,Target_rate,cum_Target,cum_nonTarget,cum_pct_Total,KS
Bin,,,,,,,,,,
1,3053,0.582785,0.898824,0.694005,2136,69.964%,31.01%,3.88%,10.00%,27.14
2,3053,0.375109,0.582709,0.475383,1395,45.693%,51.27%,10.89%,20.00%,40.38
3,3053,0.252737,0.375096,0.308172,899,29.446%,64.32%,20.00%,30.00%,44.32
4,3053,0.183742,0.252720,0.215114,622,20.373%,73.36%,30.29%,40.00%,43.07
5,3052,0.141169,0.183675,0.160485,503,16.481%,80.66%,41.07%,50.00%,39.59
6,3053,0.111910,0.141162,0.125469,414,13.560%,86.67%,52.23%,60.00%,34.44
7,3053,0.095339,0.111908,0.103523,304,9.957%,91.08%,63.86%,70.00%,27.23
8,3053,0.071246,0.095321,0.084184,276,9.040%,95.09%,75.60%,80.00%,19.49
9,3053,0.056579,0.071237,0.063413,201,6.584%,98.01%,87.67%,90.00%,10.34


In [247]:
from sklearn.metrics import classification_report
print(classification_report(Y_train,clf.predict(X_train_c)))

              precision    recall  f1-score   support

       False       0.84      0.94      0.89     55057
        True       0.67      0.40      0.50     16177

    accuracy                           0.82     71234
   macro avg       0.76      0.67      0.70     71234
weighted avg       0.80      0.82      0.80     71234



In [248]:
print(classification_report(Y_test,clf.predict(X_test_c)))

              precision    recall  f1-score   support

       False       0.84      0.94      0.89     23642
        True       0.65      0.40      0.49      6887

    accuracy                           0.82     30529
   macro avg       0.75      0.67      0.69     30529
weighted avg       0.80      0.82      0.80     30529



In [250]:
features=pd.DataFrame({'Feature':selected_cols,'Importance':clf.feature_importances_}).sort_values(by="Importance",ascending=False).reset_index(drop=True)

In [251]:
importance_cols=list(features[features['Importance']>500]['Feature'])

In [252]:
import xgboost as xgb

In [293]:
xg=xgb.XGBClassifier(importance_type='gain',n_estimators=50,max_depth=5)

In [288]:
xg.fit(X_train_c,Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type='gain',
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=50, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [289]:
y_pred=xg.predict_proba(X_train_c)[:,1]

In [290]:
ks(Y_train,y_pred)

KS is 47.72


,Total_Count,min_prob,max_prob,mean_prob,Target,Target_rate,cum_Target,cum_nonTarget,cum_pct_Total,KS
Bin,,,,,,,,,,
1,7124,0.582134,0.966434,0.707118,5353,75.140%,33.09%,3.22%,10.00%,29.87
2,7123,0.374313,0.582109,0.469998,3307,46.427%,53.53%,10.15%,20.00%,43.39
3,7123,0.249262,0.374298,0.304849,2159,30.310%,66.88%,19.16%,30.00%,47.72
4,7124,0.182187,0.249245,0.214140,1512,21.224%,76.23%,29.36%,40.00%,46.87
5,7123,0.139796,0.182154,0.159236,1144,16.061%,83.30%,40.22%,50.00%,43.08
6,7123,0.112195,0.139796,0.124986,868,12.186%,88.66%,51.58%,60.00%,37.09
7,7124,0.094098,0.112195,0.103195,708,9.938%,93.04%,63.23%,70.00%,29.81
8,7123,0.067698,0.094080,0.080646,512,7.188%,96.20%,75.24%,80.00%,20.97
9,7123,0.053163,0.067698,0.060178,383,5.377%,98.57%,87.48%,90.00%,11.09


In [291]:
y_pred_test=xg.predict_proba(X_test_c)[:,1]

In [292]:
ks(Y_test,y_pred_test)

KS is 44.379999999999995


,Total_Count,min_prob,max_prob,mean_prob,Target,Target_rate,cum_Target,cum_nonTarget,cum_pct_Total,KS
Bin,,,,,,,,,,
1,3053,0.586223,0.982641,0.708531,2125,69.604%,30.86%,3.93%,10.00%,26.93
2,3053,0.374617,0.586214,0.473716,1390,45.529%,51.04%,10.96%,20.00%,40.08
3,3053,0.251647,0.374567,0.307067,918,30.069%,64.37%,19.99%,30.00%,44.38
4,3053,0.185488,0.251620,0.216696,600,19.653%,73.08%,30.37%,40.00%,42.71
5,3052,0.141613,0.185480,0.161442,508,16.645%,80.46%,41.13%,50.00%,39.33
6,3053,0.112650,0.141611,0.125864,419,13.724%,86.54%,52.27%,60.00%,34.27
7,3053,0.094844,0.112650,0.103702,310,10.154%,91.04%,63.87%,70.00%,27.17
8,3053,0.068196,0.094844,0.081113,270,8.844%,94.96%,75.64%,80.00%,19.32
9,3053,0.053137,0.068196,0.060255,196,6.420%,97.81%,87.73%,90.00%,10.08


In [314]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
level0 = list()
level0.append(('lr', LogisticRegression()))
level0.append(('knn', KNeighborsClassifier()))
level0.append(('cart', DecisionTreeClassifier()))
level0.append(('svm', SVC()))
level0.append(('bayes', GaussianNB()))
# define meta learner model
level1 = LogisticRegression()
# define the stacking ensemble
model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
# fit the model on all available data

In [315]:
model.fit(X_train_c,Y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

StackingClassifier(cv=5,
                   estimators=[('lr', LogisticRegression()),
                               ('knn', KNeighborsClassifier()),
                               ('cart', DecisionTreeClassifier()),
                               ('svm', SVC()), ('bayes', GaussianNB())],
                   final_estimator=LogisticRegression())

In [316]:
from sklearn.metrics import classification_report
print(classification_report(Y_train,model.predict(X_train_c)))

              precision    recall  f1-score   support

       False       0.84      0.97      0.90     55057
        True       0.79      0.35      0.49     16177

    accuracy                           0.83     71234
   macro avg       0.81      0.66      0.69     71234
weighted avg       0.83      0.83      0.81     71234



In [317]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,model.predict(X_test_c)))

              precision    recall  f1-score   support

       False       0.82      0.96      0.89     23642
        True       0.68      0.30      0.41      6887

    accuracy                           0.81     30529
   macro avg       0.75      0.63      0.65     30529
weighted avg       0.79      0.81      0.78     30529



In [184]:
from sklearn.ensemble import AdaBoostClassifier
clf=AdaBoostClassifier(random_state=50,n_estimators=250,learning_rate=0.1)

In [185]:
clf.fit(X_train_c,Y_train)

AdaBoostClassifier(learning_rate=0.1, n_estimators=250, random_state=50)

In [186]:
from sklearn.metrics import classification_report
print(classification_report(Y_train,clf.predict(X_train_c)))

              precision    recall  f1-score   support

       False       0.84      0.94      0.89     55057
        True       0.65      0.39      0.49     16177

    accuracy                           0.81     71234
   macro avg       0.75      0.67      0.69     71234
weighted avg       0.80      0.81      0.80     71234



In [187]:
print(classification_report(Y_test,clf.predict(X_test_c)))

              precision    recall  f1-score   support

       False       0.84      0.94      0.89     23642
        True       0.65      0.40      0.49      6887

    accuracy                           0.81     30529
   macro avg       0.74      0.67      0.69     30529
weighted avg       0.80      0.81      0.80     30529



In [188]:
from sklearn.neural_network import MLPClassifier

In [214]:
clf = MLPClassifier(solver='adam', alpha=0.05,hidden_layer_sizes=(15,), random_state=1,activation='relu',
                    learning_rate='adaptive')

In [215]:
clf.fit(X_train_c,Y_train)

MLPClassifier(alpha=0.05, hidden_layer_sizes=(15,), learning_rate='adaptive',
              random_state=1)

In [216]:
y_pred=clf.predict_proba(X_train_c)[:,1]

In [217]:
ks(y_pred,Y_train)

KS is 30.490000000000002


,Total_Count,min_prob,max_prob,mean_prob,Target,Target_rate,cum_Target,cum_nonTarget,cum_pct_Total,KS
Bin,,,,,,,,,,
1,7123,True,True,1.000000,2917,40.955%,22.78%,7.20%,10.00%,15.58
2,7124,True,True,1.000000,2847,39.973%,45.01%,14.52%,20.00%,30.49
3,7124,False,False,0.000000,816,11.455%,51.38%,25.31%,30.00%,26.06
4,7123,False,False,0.000000,818,11.496%,57.77%,36.10%,40.00%,21.67
5,7123,False,False,0.000000,810,11.373%,64.10%,46.91%,50.00%,17.19
6,7124,False,False,0.000000,807,11.334%,70.40%,57.72%,60.00%,12.68
7,7123,False,False,0.000000,817,11.477%,76.78%,68.51%,70.00%,8.27
8,7123,False,False,0.000000,784,11.009%,82.91%,79.36%,80.00%,3.54
9,7124,False,False,0.000000,816,11.465%,89.28%,90.16%,90.00%,0.88


In [218]:
from sklearn.metrics import classification_report
print(classification_report(Y_train,clf.predict(X_train_c)))

              precision    recall  f1-score   support

       False       0.84      0.93      0.88     55057
        True       0.63      0.40      0.49     16177

    accuracy                           0.81     71234
   macro avg       0.73      0.66      0.68     71234
weighted avg       0.79      0.81      0.79     71234



In [219]:
print(classification_report(Y_test,clf.predict(X_test_c)))

              precision    recall  f1-score   support

       False       0.84      0.93      0.88     23642
        True       0.62      0.40      0.49      6887

    accuracy                           0.81     30529
   macro avg       0.73      0.66      0.68     30529
weighted avg       0.79      0.81      0.79     30529



In [108]:
df_test=pd.read_csv('test.csv',encoding='latin1')

In [110]:
df_test['defects']=clf.predict_proba(df_test[cols])[:,1]

In [111]:
df_submit=df_test[['id','defects']]

In [112]:
df_submit

,id,defects
0,101763,0.268611
1,101764,0.218304
2,101765,0.675091
3,101766,0.472892
4,101767,0.142119
...,...,...
67837,169600,0.274340
67838,169601,0.115581
67839,169602,0.174707
67840,169603,0.099954


In [113]:
df_submit.to_csv('Submission.csv',index=False)